In [3]:
import time

import numpy as np
import os
import tensorflow as tf
import matplotlib.pyplot as plt
import cv2 as cv
import random
## keras stuff
import keras
from keras.models import Sequential
from keras.layers.normalization import BatchNormalization
from keras.layers import Dense, Input, Activation, \
                    Dropout, Flatten, Conv2D, MaxPooling2D

from keras.layers.core import Flatten, Dense, Dropout
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import Convolution2D,Conv2D, MaxPooling2D, ZeroPadding2D
from keras.constraints import maxnorm
from keras.optimizers import SGD
from keras.preprocessing.image import ImageDataGenerator
from keras import backend as K
from keras.datasets import cifar10
from keras import backend as K
from keras import regularizers
from keras import optimizers
from keras.layers.core import Lambda
import os

os.environ['TF_ENABLE_AUTO_MIXED_PRECISION'] = '1'

Using TensorFlow backend.


## getting CIFAR-10 dataset
    helper functions

In [4]:
def unpickle(file):
    import pickle
    with open(file, 'br') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict

#returns a uint8 numpy array with m x n x color(R,G,o B)
def from_array_to_RGB(I_lin,channel_size=1024):
    L = channel_size
    side = int(np.sqrt(L)) #square image
    
    I = np.zeros((side,side,3),dtype=np.uint8)
    #==== color channels
    #R
    I[:,:,0] = I_lin[0:L].reshape((side,side)) 
    #G
    I[:,:,1] = I_lin[L:2*L].reshape((side,side))
    #B
    I[:,:,2] = I_lin[2*L:3*L].reshape((side,side))
    return I

## select_random_images_in_batch
#     Selects random images from the loaded batch, and saves them in a list.
#         1st column: image
#         2nd column: label
def select_random_images_in_batch(batch,no_of_samples=5):
    selection = random.sample(range(batch[b'data'].shape[0]), no_of_samples)
    I = []
    
    for i in range(len(selection)):
        index = selection[i]
        I.append( (from_array_to_RGB(batch[b'data'][index], \
                                   batch[b'data'][index].size//3),\
                                   batch[b'labels'][index], index) )
               
    return I

## display 25 random images from CIFAR dataset
    
       this selects 5 random images from each batch_n file, n = 1..5
       
       batches is a Python list that holds dictionaries:
            batches[0][b'data] returns 10,000 horizontalized images.
            batches[0][b'data][0] is the 1st image in the dataset

In [6]:
%matplotlib
path = os.getcwd() + "\\cifar-10-batches-py\\"
file = []
batches = []

# file = ['data_batch_{}'.format(i) for i in range(5)]
for i in range(5): #getting path + name of the folder for the dataset
    batches.append(unpickle(path + "data_batch_" + str(i+1)))
batches.append(unpickle( path + "test_batch" )) 

m = 5
no_of_samples = m**2
I_list = []
for i in range(m):
    I_list = select_random_images_in_batch(batches[i], no_of_samples)
    for j in range(no_of_samples):
        plt.subplot(m,m,j+1)
        plt.imshow(I_list[j][0])
        plt.show

Using matplotlib backend: Qt5Agg


## preprocessing dataset
    putting all batches together in one variable

In [1]:
num_of_classes = 10

X_train, X_test = [],[]
t_train, t_test = [],[]

k = 0
for i in range(len(batches)):
    for j in range(10_000):
        if i == (len(batches)-1): #test set
            X_test.append(from_array_to_RGB(batches[i][b'data'][j], \
                               batches[i][b'data'][j].size//3))
            t_test.append(batches[i][b'labels'][j])        
        else: #train test
            X_train.append(from_array_to_RGB(batches[i][b'data'][j], \
                               batches[i][b'data'][j].size//3))
            t_train.append(batches[i][b'labels'][j])
   
#another option is to download the dataset with this line of code
# (x_train, y_train), (x_test, y_test) = cifar10.load_data() 

#binarize labels for use with VGG16
t_train = keras.utils.np_utils.to_categorical(t_train, num_of_classes)
t_test = keras.utils.np_utils.to_categorical(t_test, num_of_classes)

#normalized by dividing byt the max
X_train = np.asarray(X_train,dtype=np.float32)/255
X_train = X_train.astype('float16')

X_test = np.asarray(X_test,dtype=np.float32)/255
X_test = X_test.astype('float16')

NameError: name 'batches' is not defined

## calculations for stride/filter size and image size match

In [ ]:
stride = lambda W,F,P,S: (W-F+2*P)/S + 1 
#1. cond2d layer
print(32)
NEW_W = stride(32,4,0,2)
# print(NEW_W)
#1. maxpooling layer
NEW_W = stride(NEW_W,3,0,1)
print(NEW_W)
#####################
#2. cond2d layer
NEW_W = stride(NEW_W,3,1,1)
# print(NEW_W)
#2. maxpooling layer
NEW_W = stride(NEW_W,3,0,1)
print(NEW_W)
#####################
#3. cond2d layer
NEW_W = stride(NEW_W,3,1,1)
# print(NEW_W)
#3. maxpooling layer
NEW_W = stride(NEW_W,3,0,1)
print(NEW_W)
#####################

## parameters for CNN

In [ ]:
batch_size = 500
num_classes = 10
X_shape = [32,32,3]

learning_rate = 0.1
lr_decay = 1e-6
lr_drop = 20
maxepochs = 3

## simple AlexNet - (very small) deep CNN

In [ ]:
sgd = optimizers.SGD(lr=learning_rate, decay=lr_decay, momentum=0.9, nesterov=True)
def alexsoaresnet(no_outputs=10,weights_path=None):
    weight_decay = 1e-6
    model = Sequential()
####################
#     model.add(keras.layers.ZeroPadding2D(padding=(1, 1)))
    model.add(Conv2D(48, (4, 4), strides=2, padding='valid', \
                     input_shape=(32,32,3), \
                     kernel_regularizer=regularizers.l2(weight_decay)))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2), strides=1))
####################
    model.add(keras.layers.ZeroPadding2D(padding=(1, 1)))
    model.add(Conv2D(192, (3, 3), padding='valid', \
                     input_shape=(13,13,24), \
                     kernel_regularizer=regularizers.l2(weight_decay)))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(3, 3), strides=1))
####################
    model.add(keras.layers.ZeroPadding2D(padding=(1, 1)))
    model.add(Conv2D(128, (3, 3), padding='valid', \
                     input_shape=(11,11,96), \
                     kernel_regularizer=regularizers.l2(weight_decay)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(3, 3), strides=1))
####################
    model.add(Flatten())
    model.add(Dense(256,kernel_regularizer=regularizers.l2(weight_decay)))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.5))
####################
    model.add(Dense(no_outputs))
    model.add(Activation('softmax'))    
    
    if weights_path:
        model.load_weights(weights_path, by_name=True)
    
    model.compile(loss='categorical_crossentropy',\
                  optimizer=sgd, metrics=['accuracy'])
    
    return model

def lr_scheduler(epoch):
    return learning_rate * (0.5 ** (epoch // lr_drop))

reduce_lr = keras.callbacks.LearningRateScheduler(lr_scheduler)

  ## image settings for running the network

In [ ]:
datagen = ImageDataGenerator(    
    featurewise_center=False,  # set input mean to 0 over the dataset
    samplewise_center=False,  # set each sample mean to 0
    featurewise_std_normalization=False,  # divide inputs by std of the dataset
    samplewise_std_normalization=False,  # divide each input by its std
    zca_whitening=False,  # apply ZCA whitening
    rotation_range=15,  # randomly rotate images in the range (degrees, 0 to 180)
    width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
    height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
    horizontal_flip=True,  # randomly flip images
    vertical_flip=False)  # randomly flip images
# (std, mean, and principal components if ZCA whitening is applied).

## compiling and running the network
    commented out by default because it takes long to run

In [ ]:
# as_net = alexsoaresnet()
# as_net.summary()

# alexSnet = as_net.fit_generator(datagen.flow(X_train, t_train,\
#                                         batch_size=batch_size),\
#                            steps_per_epoch=X_train.shape[0] // batch_size,\
#                            epochs=maxepochs, validation_data=(X_test, t_test),\
#                            callbacks=[reduce_lr], verbose=1)

## saving weights and plotting trainning stats

In [ ]:
# as_net.save_weights('transfer_cifar10alexSnet83.h5')
# as_net.save('alexSnet.h5')  # creates a HDF5 file 'my_model.h5'
# del model  # deletes the existing model

plt.figure(0)
plt.plot(alexSnet.history['acc'],'r')
plt.plot(alexSnet.history['val_acc'],'g')
plt.xticks(np.arange(0, 51, 2.0))
plt.rcParams['figure.figsize'] = (8, 6)
plt.xlabel("Num of Epochs")
plt.ylabel("Accuracy")
plt.title("Training Accuracy vs Validation Accuracy")
plt.legend(['train','validation'])
 
plt.figure(1)
plt.plot(alexSnet.history['loss'],'r')
plt.plot(alexSnet.history['val_loss'],'g')
plt.xticks(np.arange(0, 51, 2.0))
plt.rcParams['figure.figsize'] = (8, 6)
plt.xlabel("Num of Epochs")
plt.ylabel("Loss")
plt.title("Training Loss vs Validation Loss")
plt.legend(['train','validation'])
plt.show()
scores = as_net.evaluate(X_test, t_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

## loading merch dataset
    labels are:
    1. "Cap"
    2. "Cube"
    3. "Playing_Cards"
    4. "Screwdriver"
    5. "Torch"
    which are redefined as binary later

In [ ]:
no_classes = 5
merch_labels = ["Cap", "Cube", "Playing_Cards", "Screwdriver", "Torch"]
merch_labels_numerical = [0, 1, 2, 3, 4]
merch_data_paths = {}

for i in range(len(merch_labels)):
    merch_data_paths[merch_labels[i]] = os.getcwd() + "\\MerchData\\" + merch_labels[i]
    
#retrieves all files in folder
X = []
T = []
for i in range(len(merch_labels)): #goes beyond to match labels
    data_path = merch_data_paths[merch_labels[i-1]]
#     merch_data_paths["Cap"]
    file_names = os.listdir(data_path)
    for j in range(len(file_names)):
        file_path = data_path + "\\" + file_names[j]
        img = cv.resize(cv.imread(file_path), dsize=(32,32))
        #img is resized to match the networks' input sizee
        X.append(img)
        T.append(i) #label

## pre processing

In [ ]:
X = (np.asarray(X,dtype=np.float32)/255).astype('float16')
T = np.asarray(T,dtype=np.float32)
no_samples_train = 45

indeces = np.random.permutation(X.shape[0])
idx_train, idx_test = indeces[:no_samples_train], indeces[no_samples_train:]
#samples
X_train2 = X[idx_train]
X_test2 = X[idx_test]
#targets
t_train2 = keras.utils.np_utils.to_categorical(T[idx_train], no_classes)
t_test2 = keras.utils.np_utils.to_categorical(T[idx_test], no_classes)

## Loading pre-trained CNN
    trained on the CIFAR-10 dataset and freezing top layers
    number of epochs was much higer than the suggested by the assingment
    (I had access to a GPU with 12 GB of VRAM)

In [ ]:
from keras.models import load_model
if 'alexSnet' in globals():
    del alexSnet

weight_decay = 1e-6
alexSnet = load_model('alexSnet\\alexSnet.h5')

for i in range(len(alexSnet.layers)):
    alexSnet.layers[i].trainable = False
# for i in range(0,-7,-1):
#     alexSnet.layers[i].trainable = True
for i in range(6):
    alexSnet.pop() #removes last two layers

alexSnet.add(Dense(128,kernel_regularizer=regularizers.l2(weight_decay)))
alexSnet.layers[-1].trainable = True
alexSnet.add(Activation('relu'))
alexSnet.add(BatchNormalization())
alexSnet.add(Dropout(0.5))
alexSnet.add(Dense(no_classes))
alexSnet.layers[-1].trainable = True
alexSnet.add(Activation('softmax'))


alexSnet.compile(loss='categorical_crossentropy',\
                 optimizer=sgd, metrics=['accuracy'])

## compiling and running the network(again!)

In [ ]:
epochs_merch = 200
batch_size = 10


alexSnet.summary()
# for i in range(len(VGG_NN.layers)):
#     VGG_NN.layers[i].trainable = False
cnn = alexSnet.fit_generator(datagen.flow(X_train2, t_train2,\
                                        batch_size=batch_size),\
                           steps_per_epoch=X_train2.shape[0] // batch_size,\
                           epochs=epochs_merch, validation_data=(X_test2, t_test2),\
                           callbacks=[reduce_lr], verbose=1)

## check performance of transfer learning

In [ ]:
# as_net.save_weights('transfer_cifar10alexSnet83.h5')

# # del model  # deletes the existing model

plt.figure(0)
plt.plot(cnn.history['acc'],'r')
plt.plot(cnn.history['val_acc'],'g')
plt.xticks(np.arange(0, epochs_merch+1, epochs_merch//10))
plt.rcParams['figure.figsize'] = (8, 6)
plt.xlabel("Num of Epochs")
plt.ylabel("Accuracy")
plt.title("transfer learning - train accuracy vs validation accuracy")
plt.legend(['train','validation'])
 
plt.figure(1)
plt.plot(cnn.history['loss'],'r')
plt.plot(cnn.history['val_loss'],'g')
plt.xticks(np.arange(0, epochs_merch+1, epochs_merch//10))
plt.rcParams['figure.figsize'] = (8, 6)
plt.xlabel("Num of Epochs")
plt.ylabel("Loss")
plt.title("transfer learning - train Loss vs validation Loss")
plt.legend(['train','validation'])
plt.show()
scores = alexSnet.evaluate(X_test2, t_test2, verbose=0)
print("Test Accuracy: %.2f%%" % (scores[1]*100))

In [ ]:
# alexSnet.save('alexSnet_transfer_merch.h5')  # creates a HDF5 file 'my_model.h5'

## training from scratch on the Merch dataset
    the function is copied here again to prevent refeences to the same memory location for the model

In [ ]:
sgd = optimizers.SGD(lr=learning_rate, decay=lr_decay, momentum=0.9, nesterov=True)
def alexsoaresnet2(no_outputs=10,weights_path=None):
    weight_decay = 1e-6
    model = Sequential()
####################
#     model.add(keras.layers.ZeroPadding2D(padding=(1, 1)))
    model.add(Conv2D(48, (4, 4), strides=2, padding='valid', \
                     input_shape=(32,32,3), \
                     kernel_regularizer=regularizers.l2(weight_decay)))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2), strides=1))
####################
    model.add(keras.layers.ZeroPadding2D(padding=(1, 1)))
    model.add(Conv2D(192, (3, 3), padding='valid', \
                     input_shape=(13,13,24), \
                     kernel_regularizer=regularizers.l2(weight_decay)))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(3, 3), strides=1))
####################
    model.add(keras.layers.ZeroPadding2D(padding=(1, 1)))
    model.add(Conv2D(128, (3, 3), padding='valid', \
                     input_shape=(11,11,96), \
                     kernel_regularizer=regularizers.l2(weight_decay)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(3, 3), strides=1))
####################
    model.add(Flatten())
    model.add(Dense(256,kernel_regularizer=regularizers.l2(weight_decay)))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.5))
####################
    model.add(Dense(no_outputs))
    model.add(Activation('softmax'))    
    
    if weights_path:
        model.load_weights(weights_path, by_name=True)
    
    model.compile(loss='categorical_crossentropy',\
                  optimizer=sgd, metrics=['accuracy'])
    
    return model

def lr_scheduler(epoch):
    return learning_rate * (0.5 ** (epoch // lr_drop))

reduce_lr = keras.callbacks.LearningRateScheduler(lr_scheduler)

In [ ]:
aleSnet_merch = alexsoaresnet2(no_outputs=5)
epochs_merch = 200
batch_size = 10

aleSnet_merch.summary()
# for i in range(len(VGG_NN.layers)):
#     VGG_NN.layers[i].trainable = False
aleSnet_merch_CNN = alexSnet.fit_generator(datagen.flow(X_train2, t_train2,\
                                        batch_size=batch_size),\
                           steps_per_epoch=X_train2.shape[0] // batch_size,\
                           epochs=epochs_merch, validation_data=(X_test2, t_test2),\
                           callbacks=[reduce_lr], verbose=1)

## checking performance of training form scratch

In [ ]:
# as_net.save_weights('transfer_cifar10alexSnet83.h5')
# # del model  # deletes the existing model

plt.figure(0)
plt.plot(aleSnet_merch_CNN.history['acc'],'r')
plt.plot(aleSnet_merch_CNN.history['val_acc'],'g')
plt.xticks(np.arange(0, epochs_merch+1, epochs_merch//10))
plt.rcParams['figure.figsize'] = (8, 6)
plt.xlabel("Num of Epochs")
plt.ylabel("Accuracy")

plt.title("training from scratch: train accuracy vs validation accuracy")
plt.legend(['train','validation'])
 
plt.figure(1)
plt.plot(aleSnet_merch_CNN.history['loss'],'r')
plt.plot(aleSnet_merch_CNN.history['val_loss'],'g')
plt.xticks(np.arange(0, epochs_merch+1, epochs_merch//10))
plt.rcParams['figure.figsize'] = (8, 6)
plt.xlabel("Num of Epochs")
plt.ylabel("Loss")
plt.title("training from scratch: train Loss vs validation Loss")
plt.legend(['train','validation'])
plt.show()
scores = aleSnet_merch.evaluate(X_test2, t_test2, verbose=0)
print("Test Accuracy: %.2f%%" % (scores[1]*100))

In [ ]:
# aleSnet_merch.predict(X_test)

In [ ]:
aleSnet_merch.evaluate(X_test2, t_test2, verbose=1)